In [1]:
import torch
from cora_loader import CitationNetwork,CocitationNetwork,ConfigurationModelCitationNetwork
from multi_layered_model import MonoModel,BiModel,TriModel
from torch_geometric.nn import GCNConv,SAGEConv
import time
import torch.nn.functional as F
import copy

In [2]:
cora = CitationNetwork('/tmp/cora2','cora',directed=False)

Processing...
Read features: RUNNING
Read features: DONE
Read edges: RUNNING
10858
Read edges: DONE
False
10858
10858
Done!


In [4]:
citeseer = CitationNetwork('/tmp/citeseer','citeseer',directed=False)
PubMed = CitationNetwork('/tmp/PubMed','PubMed',directed=False)

Processing...
Read features: RUNNING
Read features: DONE
Read edges: RUNNING
9182
Read edges: DONE
False
18364
9182


AssertionError: 

In [3]:

def run_and_eval_model(dataset,channels,modelType,lr,weight_decay,architecture=MonoModel,epochs=200):
    # training process (without batches/transforms)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = architecture(modelType,dataset,channels).to(device)
#     print(model)
    data = dataset[0].to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr=lr,weight_decay=weight_decay)
    model.train() # to enter training phase
    maxacc=0
    chosen = None
    for epoch in range(epochs):
        optimizer.zero_grad() # saw this a lot in the beginning, maybe resetting gradients (not to accumulate)
        out = model(data) # this calls the forward method apparently
        loss = F.nll_loss(out[data.train_mask],data.y[data.train_mask]) # nice indexing, easy and short
        loss.backward() # magic: real back propagation step, takes care of the gradients and stuff
        optimizer.step() # maybe updates the params to be optimized
        
        model.eval()
        _,pred = model(data).max(dim=1) # take prediction out of softmax
        correct = float(pred[data.val_mask].eq(data.y[data.val_mask]).sum().item())
        acc = correct / data.val_mask.sum().item()
        if acc > maxacc:
            maxacc = acc
            chosen=copy.deepcopy(model)
        model.train()
    chosen.eval() # enter eval phase
    _,pred = chosen(data).max(dim=1) # take prediction out of softmax
    correct = float(pred[data.val_mask].eq(data.y[data.val_mask]).sum().item())
    acc = correct / data.val_mask.sum().item()
    return acc

def eval_multiple(dataset,channels,modelType,architecture=MonoModel,lr=0.1,wd=0.01,runs=100,epochs=200):
    best_acc = 0
#     for lr in [1e-2,1e-1,1]:
#         for wd in [1e-4,1e-3,1e-2,1e-1]:
    for lr in [0.01]:
        for wd in [5e-4]:
            start = time.time()
            accs = []
            for i in range(runs):
                accs.append(run_and_eval_model(dataset,channels,modelType,lr,wd,architecture,epochs))
            acc = sum(accs)/len(accs)
            print("lr={:.4f}\twd={:.5f}\t{:.4f}".format(lr,wd,acc))
            if acc > best_acc:
                best_lr = lr
                best_wd = wd
                best_acc =acc
            elapsed_time = time.time() - start
            print('Elaplsed {}'.format(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))
    return best_lr,best_wd

In [4]:
def param_search(conv,data,channels1,channels2,channels3,outFile,runs = 10,epochs=200):
    #with open(outFile,'w') as out:
        lr,wd = eval_multiple(data,channels1,conv,runs=runs,epochs=epochs)
     #   out.write('Mono lr={:.4f}\twd={:.5f}\n'.format(lr,wd))
        print('Mono lr={:.4f}\twd={:.5f}\n'.format(lr,wd))
        lr,wd = eval_multiple(data,channels2,conv,runs=runs,epochs=epochs)
     #   out.write('Bi   lr={:.4f}\twd={:.5f}\n'.format(lr,wd))
        print('Bi   lr={:.4f}\twd={:.5f}\n'.format(lr,wd))
        lr,wd = eval_multiple(data,channels3,conv,runs=runs,epochs=epochs)
     #   out.write('Tri  lr={:.4f}\twd={:.5f}\n'.format(lr,wd))
        print(('Tri  lr={:.4f}\twd={:.5f}\n'.format(lr,wd)))

In [5]:
def SAGE(conv,data,channels1,channels2,channels3,outFile,runs = 10,epochs=200):
    #with open(outFile,'w') as out:
        lr,wd = eval_multiple(data,channels1,conv,lr=0.1,wd=0.00001,runs=runs,epochs=epochs)
     #   out.write('Mono lr={:.4f}\twd={:.5f}\n'.format(lr,wd))
        lr,wd = eval_multiple(data,channels2,conv,lr=0.1,wd=0.01,runs=runs,epochs=epochs)
     #   out.write('Bi   lr={:.4f}\twd={:.5f}\n'.format(lr,wd))
        lr,wd = eval_multiple(data,channels3,conv,lr=0.1,wd=0.01,runs=runs,epochs=epochs)
     #   out.write('Tri  lr={:.4f}\twd={:.5f}\n'.format(lr,wd))

In [6]:
def GCN(conv,data,channels1,channels2,channels3,outFile,runs = 10,epochs=200):
    with open(outFile,'w') as out:
        lr,wd = eval_multiple(data,channels1,conv,lr=0.01,wd=0.0001,runs=runs,epochs=epochs)
        out.write('Mono lr={:.4f}\twd={:.5f}\n'.format(lr,wd))
        lr,wd = eval_multiple(data,channels2,conv,lr=0.01,wd=0.0001,runs=runs,epochs=epochs)
        out.write('Bi   lr={:.4f}\twd={:.5f}\n'.format(lr,wd))
        lr,wd = eval_multiple(data,channels3,conv,lr=0.01,wd=0.0001,runs=runs,epochs=epochs)
        out.write('Tri  lr={:.4f}\twd={:.5f}\n'.format(lr,wd))

In [7]:
import warnings
warnings.filterwarnings('ignore')


In [8]:
GCN(GCNConv,cora,[128],[64],[42],'xxx.out',4)

lr=0.0100	wd=0.00050	0.7910
Elaplsed 00:00:14
lr=0.0100	wd=0.00050	0.7885
Elaplsed 00:00:08
lr=0.0100	wd=0.00050	0.7840
Elaplsed 00:00:08


In [34]:
SAGE(SAGEConv,cora,[128],[64],[42],'xxx.out',100)

lr=0.1000	wd=0.00001	0.7832
Elaplsed 00:02:15
lr=0.1000	wd=0.01000	0.7827
Elaplsed 00:02:11
lr=0.1000	wd=0.01000	0.7838
Elaplsed 00:02:10


In [17]:
param_search(SAGEConv,cora,[64],[32],[21],'SAGE_param_search_extra.out',100)

lr=0.0100	wd=0.00010	0.7803
Elaplsed 00:01:51
lr=0.0100	wd=0.00100	0.7797
Elaplsed 00:01:46
lr=0.0100	wd=0.01000	0.7821
Elaplsed 00:01:46
lr=0.0100	wd=0.10000	0.7818
Elaplsed 00:01:45
lr=0.1000	wd=0.00010	0.7832
Elaplsed 00:01:52
lr=0.1000	wd=0.00100	0.7804
Elaplsed 00:01:49
lr=0.1000	wd=0.01000	0.7818
Elaplsed 00:01:52
lr=0.1000	wd=0.10000	0.7645
Elaplsed 00:01:56
lr=1.0000	wd=0.00010	0.7801
Elaplsed 00:01:56
lr=1.0000	wd=0.00100	0.7745
Elaplsed 00:01:56
lr=1.0000	wd=0.01000	0.6452
Elaplsed 00:01:56
lr=1.0000	wd=0.10000	0.5349
Elaplsed 00:01:57
Mono lr=0.1000	wd=0.00010

lr=0.0100	wd=0.00010	0.7801
Elaplsed 00:01:56
lr=0.0100	wd=0.00100	0.7788
Elaplsed 00:01:56
lr=0.0100	wd=0.01000	0.7801
Elaplsed 00:01:56
lr=0.0100	wd=0.10000	0.7808
Elaplsed 00:01:55
lr=0.1000	wd=0.00010	0.7806
Elaplsed 00:01:55
lr=0.1000	wd=0.00100	0.7792
Elaplsed 00:01:56
lr=0.1000	wd=0.01000	0.7824
Elaplsed 00:01:55
lr=0.1000	wd=0.10000	0.7686
Elaplsed 00:01:56
lr=1.0000	wd=0.00010	0.7789
Elaplsed 00:01:56
lr=1.00

In [18]:
param_search(SAGEConv,citeseer,[64],[32],[21],'SAGE_param_search_extra.out',100)

lr=0.0100	wd=0.00010	0.7409
Elaplsed 00:01:48
lr=0.0100	wd=0.00100	0.7506
Elaplsed 00:01:49
lr=0.0100	wd=0.01000	0.7516
Elaplsed 00:01:49
lr=0.0100	wd=0.10000	0.7567
Elaplsed 00:01:47
lr=0.1000	wd=0.00010	0.7509
Elaplsed 00:01:45
lr=0.1000	wd=0.00100	0.7583
Elaplsed 00:01:46
lr=0.1000	wd=0.01000	0.7610
Elaplsed 00:01:46
lr=0.1000	wd=0.10000	0.7591
Elaplsed 00:01:47
lr=1.0000	wd=0.00010	0.7657
Elaplsed 00:01:52
lr=1.0000	wd=0.00100	0.7589
Elaplsed 00:01:51
lr=1.0000	wd=0.01000	0.7295
Elaplsed 00:01:49
lr=1.0000	wd=0.10000	0.6873
Elaplsed 00:01:44
Mono lr=1.0000	wd=0.00010

lr=0.0100	wd=0.00010	0.7338
Elaplsed 00:01:41
lr=0.0100	wd=0.00100	0.7446
Elaplsed 00:01:44
lr=0.0100	wd=0.01000	0.7489
Elaplsed 00:01:45
lr=0.0100	wd=0.10000	0.7538
Elaplsed 00:01:44
lr=0.1000	wd=0.00010	0.7457
Elaplsed 00:01:46
lr=0.1000	wd=0.00100	0.7532
Elaplsed 00:01:46
lr=0.1000	wd=0.01000	0.7594
Elaplsed 00:01:44
lr=0.1000	wd=0.10000	0.7573
Elaplsed 00:01:53
lr=1.0000	wd=0.00010	0.7604
Elaplsed 00:02:07
lr=1.00

In [8]:
def GCN(conv,data,channels1,channels2,channels3,outFile,runs = 10,epochs=200):
    #with open(outFile,'w') as out:
        lr,wd = eval_multiple(data,channels1,conv,lr=0.01,wd=0.0005,runs=runs,epochs=epochs)
     #   out.write('Mono lr={:.4f}\twd={:.5f}\n'.format(lr,wd))
        lr,wd = eval_multiple(data,channels2,conv,lr=0.01,wd=0.0005,runs=runs,epochs=epochs)
     #   out.write('Bi   lr={:.4f}\twd={:.5f}\n'.format(lr,wd))
        lr,wd = eval_multiple(data,channels3,conv,lr=0.01,wd=0.0005,runs=runs,epochs=epochs)
     #   out.write('Tri  lr={:.4f}\twd={:.5f}\n'.format(lr,wd))

wd = 0.0001

In [36]:
GCN(GCNConv,cora,[128],[64],[42],'xxx.out',100)

lr=0.0100	wd=0.00010	0.7919
Elaplsed 00:03:00
lr=0.0100	wd=0.00010	0.7915
Elaplsed 00:03:10
lr=0.0100	wd=0.00010	0.7909
Elaplsed 00:16:47


wd = 0.0005

In [39]:
GCN(GCNConv,cora,[128],[64],[42],'xxx.out',100)

lr=0.0100	wd=0.00050	0.7909
Elaplsed 00:03:31
lr=0.0100	wd=0.00050	0.7915
Elaplsed 00:03:13
lr=0.0100	wd=0.00050	0.7906
Elaplsed 00:03:11


In [22]:
param_search(GCNConv,cora,[128],[64],[42],'GCN_param_search.out',100)

lr=0.0001	wd=0.0000	0.7675
lr=0.0001	wd=0.0001	0.7666
lr=0.0001	wd=0.0010	0.7634
lr=0.0001	wd=0.0100	0.7422
lr=0.0010	wd=0.0000	0.7867
lr=0.0010	wd=0.0001	0.7871
lr=0.0010	wd=0.0010	0.7903
lr=0.0010	wd=0.0100	0.7850
lr=0.0100	wd=0.0000	0.7897
lr=0.0100	wd=0.0001	0.7919
lr=0.0100	wd=0.0010	0.7913
lr=0.0100	wd=0.0100	0.7874
lr=0.1000	wd=0.0000	0.7774
lr=0.1000	wd=0.0001	0.7755
lr=0.1000	wd=0.0010	0.7779
lr=0.1000	wd=0.0100	0.7867
Elaplsed 00:03:04
Mono lr=0.0100	wd=0.00010
lr=0.0001	wd=0.0000	0.7367
lr=0.0001	wd=0.0001	0.7313
lr=0.0001	wd=0.0010	0.7367
lr=0.0001	wd=0.0100	0.7111
lr=0.0010	wd=0.0000	0.7865
lr=0.0010	wd=0.0001	0.7860
lr=0.0010	wd=0.0010	0.7880
lr=0.0010	wd=0.0100	0.7864
lr=0.0100	wd=0.0000	0.7878
lr=0.0100	wd=0.0001	0.7918
lr=0.0100	wd=0.0010	0.7906
lr=0.0100	wd=0.0100	0.7882
lr=0.1000	wd=0.0000	0.7820
lr=0.1000	wd=0.0001	0.7750
lr=0.1000	wd=0.0010	0.7788
lr=0.1000	wd=0.0100	0.7859
Elaplsed 00:02:57
Bi   lr=0.0100	wd=0.00010
lr=0.0001	wd=0.0000	0.6966
lr=0.0001	wd=0.0001	0